In [2]:
%matplotlib inline
import networkx as nx
import numpy as np 
import pandas as pd 
import itertools


In [3]:
import numpy as np
import networkx as nx
from networkx.algorithms import community #use community algorithm to detec communities within a graph



FielName="email-Eu-core-temporal.txt"
Graphtype=nx.DiGraph()   # use net.Graph() for undirected graph

# How to read from a file. Note: if your egde weights are int, change float to int.
G = nx.read_edgelist(FielName, create_using=Graphtype, nodetype=int, data=(('weight',float),))

get_nodes = False
if get_nodes == True:
    # Find the total number of degree, in_degree and out_degree for each node
    for x in G.nodes():
          #print ("Node: ", x, " has total #degree: ",G.degree(x), " , In_degree: ", G.out_degree(x)," and out_degree: ", G.in_degree(x))
        print(1)
get_edges = False
if get_edges == True:
    # Find the weight for each node
    for u,v in G.edges():
        #print ("Weight of Edge ("+str(u)+","+str(v)+")", G.get_edge_data(u,v))
        print(1)



# print(sorted(nx.get_edge_attributes(G, 'weight')))

In [ ]:
#Plot graph 
#
#import matplotlib.pyplot as plt

#nx.draw(G)
#plt.draw()
#plt.show()

In [4]:
#read as pandas df
df = pd.read_csv(FielName, sep = " ", header = None)
df.columns = ["start_node", "end_node", "time_stamp"]

In [5]:
#remove edges with timestamp larger than average over all timestamp 
#use the subset after removing edges as the training data to predict links 
#compare the prediction result to the edges of full dataset. 
#mean = df["time_stamp"].mean()

def train_test_split(df, percentage_of_train):
    mean = np.percentile( df["time_stamp"], percentage_of_train)
    location = (df['time_stamp'] < mean)
    sub_df = df.loc[location]
    location_test = (df['time_stamp'] > mean)
    sub_test = df.loc[location_test]
    sub_test.drop_duplicates(subset=['start_node','end_node'], inplace = True)

    #build a subgraph from dataframe created 
    G_sub=nx.from_pandas_edgelist(sub_df, "start_node", "end_node", ['time_stamp'])

    #convert to undirected for link prediction
    G_sub_undirected = G_sub.to_undirected()
    
    return [G_sub_undirected, sub_test]


In [6]:
def link_prediction(G_sub_undirected):
    #Preferential Attachment 
    preds = nx.preferential_attachment(G_sub_undirected, ebunch=None)
    PA = []
    for u, v, p in preds:
        PA.append([u,v,p])

    
    #Resource Allocation Index 
    preds_1 = nx.resource_allocation_index(G_sub_undirected, ebunch=None)
    RA = []
    for u, v, p in preds_1:
        RA.append([u,v,p])

    #Adamic-Adar Index
    preds_2 = nx.adamic_adar_index(G_sub_undirected, ebunch=None)
    AAI = []
    for u, v, p in preds_2:
        AAI.append([u,v,p])
    
    #Jaccard_coefficient
    preds_3 = nx.jaccard_coefficient(G_sub_undirected, ebunch=None)
    JC = []
    for u, v, p in preds_3:
        JC.append([u,v,p])
    #print(PA)
        
    return [PA, RA, AAI, JC]
    
#Number of Common Neighbor - removed since it require community information 
#preds_4 = nx.cn_soundarajan_hopcroft(G_sub_undirected, ebunch=None)
#CSH= []
#for u, v, p in preds_4:
   # AAI.append([u,v,p])
    
#Resource Allocation Index - removed since it require community information 
#preds_5 = nx.ra_index_soundarajan_hopcroft(G_sub_undirected, ebunch=None)
#RAI= []
#for u, v, p in preds_5:
    #RAI.append([u,v,p])
    
# Ratio of witin and inter cluster- removed since it require community information 
#preds_6 = nx.ra_index_soundarajan_hopcroft(G_sub_undirected, ebunch=None)
#WIC= []
#for u, v, p in preds_6:
   # WIC.append([u,v,p])

    



In [35]:
def evaluation(link_list, sub_test, percentile):
    i = 0
    while i < 4: 
        df_result = pd.DataFrame(link_list[i])
        df_result.columns = ['start_node', 'end_node', 'score']  
        df_result = df_result.drop(df_result[df_result.score == 0].index)
        #print(np.min(df_result["score"]))
        threshold = np.percentile( df_result["score"], percentile)
        if i == 0:
            print("Preferential Attachment:")
        if i == 1: 
            print("Resource Allocation Index: ")
        if i == 2: 
            print("Adamic-adar Index: ")
        if i == 3: 
            print("Jaccard_coefficient: ")
       
        print("Current Threshold Percentile = ", percentile)
        print("Current Threshold Value = ", threshold)
        link_location = (df_result['score'] > threshold)
        df_link = df_result.loc[link_location]
        number_edge_in_test = sub_test.shape[0]
        number_edge_in_prediction = df_link.shape[0]
        df_merged = pd.merge(df_link, sub_test , how='inner', on=['start_node','end_node'])
        df_merged.drop_duplicates(subset=['start_node','end_node'], inplace = True)
        number_edge_in_both =df_merged.shape[0]
        if number_edge_in_prediction == 0:
            print("No links predicted")
            print("----------------------------------------------------------------------------")
            i = i + 1
        else:
            print("number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: ")
            print(number_edge_in_test,number_edge_in_prediction,number_edge_in_both)
            #false positive - prediction: exist, Actual: does not exist 
            print("False Positive Rate: ", (number_edge_in_prediction - number_edge_in_both)/number_edge_in_prediction)
            #false negative - prediction: does not exist, Actual: exist 
            print("False Positive Rate: ", (number_edge_in_test - number_edge_in_both)/number_edge_in_test)
            #Accuracy - percentage of links exsit in original data and is predicted by algorithm
            print("Accuracy: ", number_edge_in_both/number_edge_in_test)
            print("----------------------------------------------------------------------------")
            i = i + 1
    


In [36]:
for x in range (5, 9):
    print("current training size is: ", x*10)
    DF = train_test_split(df, x*0.1)
    result = link_prediction(DF[0])
    for y in range (0, 9):
        evaluation(result, DF[1], y*10)
        print()
        print("*************************************************************************************")
    print('_____________________________________________________________________________________')
    print('_____________________________________________________________________________________')
    
    



current training size is:  50


C:\Users\eveli\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


1
Preferential Attachment:
Current Threshold Percentile =  0
Current Threshold Value =  1.0
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24865 105694 6987
False Positive Rate:  0.9338940715650841
False Positive Rate:  0.7190026141162277
Accuracy:  0.2809973858837724
----------------------------------------------------------------------------
0.02702702702702703
Resource Allocation Index: 
Current Threshold Percentile =  0
Current Threshold Value =  0.02702702702702703
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24865 4545 1662
False Positive Rate:  0.6343234323432343
False Positive Rate:  0.9331590589181581
Accuracy:  0.06684094108184195
----------------------------------------------------------------------------
0.2769378934088574
Adamic-adar Index: 
Current Threshold Percentile =  0
Current Threshold Value =  0.2769378934088574
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24865 4545 1662
False Positive 

1
Preferential Attachment:
Current Threshold Percentile =  50
Current Threshold Value =  6.0
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24865 49835 4766
False Positive Rate:  0.9043644025283435
False Positive Rate:  0.8083249547556807
Accuracy:  0.19167504524431933
----------------------------------------------------------------------------
0.02702702702702703
Resource Allocation Index: 
Current Threshold Percentile =  50
Current Threshold Value =  0.0625
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24865 2416 1029
False Positive Rate:  0.5740894039735099
False Positive Rate:  0.9586165292579931
Accuracy:  0.041383470742006837
----------------------------------------------------------------------------
0.2769378934088574
Adamic-adar Index: 
Current Threshold Percentile =  50
Current Threshold Value =  0.36067376022224085
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24865 2416 1029
False Positive Rate:  0

24844 4746 1726
False Positive Rate:  0.6363253265908133
False Positive Rate:  0.9305264852680728
Accuracy:  0.06947351473192723
----------------------------------------------------------------------------
0.2749075777424936
Adamic-adar Index: 
Current Threshold Percentile =  10
Current Threshold Value =  0.28126641406272834
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24844 4746 1726
False Positive Rate:  0.6363253265908133
False Positive Rate:  0.9305264852680728
Accuracy:  0.06947351473192723
----------------------------------------------------------------------------
0.013888888888888888
Jaccard_coefficient: 
Current Threshold Percentile =  10
Current Threshold Value =  0.047619047619047616
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24844 5104 1761
False Positive Rate:  0.6549764890282131
False Positive Rate:  0.929117694413138
Accuracy:  0.07088230558686202
---------------------------------------------------------------------

24844 2167 923
False Positive Rate:  0.5740655283802492
False Positive Rate:  0.9628481725970053
Accuracy:  0.037151827402994685
----------------------------------------------------------------------------
0.2749075777424936
Adamic-adar Index: 
Current Threshold Percentile =  60
Current Threshold Value =  0.4170323914242463
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24844 2269 1000
False Positive Rate:  0.5592772146319964
False Positive Rate:  0.9597488327161487
Accuracy:  0.04025116728385123
----------------------------------------------------------------------------
0.013888888888888888
Jaccard_coefficient: 
Current Threshold Percentile =  60
Current Threshold Value =  0.16666666666666663
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24844 2674 947
False Positive Rate:  0.6458489154824233
False Positive Rate:  0.9618821445821929
Accuracy:  0.038117855417807114
----------------------------------------------------------------------

number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24840 4679 1745
False Positive Rate:  0.6270570634751015
False Positive Rate:  0.9297504025764896
Accuracy:  0.07024959742351047
----------------------------------------------------------------------------
0.2749075777424936
Adamic-adar Index: 
Current Threshold Percentile =  20
Current Threshold Value =  0.31066746727980593
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24840 4679 1745
False Positive Rate:  0.6270570634751015
False Positive Rate:  0.9297504025764896
Accuracy:  0.07024959742351047
----------------------------------------------------------------------------
0.013888888888888888
Jaccard_coefficient: 
Current Threshold Percentile =  20
Current Threshold Value =  0.06666666666666667
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24840 4831 1648
False Positive Rate:  0.6588697992134134
False Positive Rate:  0.9336553945249597
Accuracy:  0.066344605475040

24840 1656 767
False Positive Rate:  0.5368357487922706
False Positive Rate:  0.9691223832528181
Accuracy:  0.030877616747181966
----------------------------------------------------------------------------
0.2749075777424936
Adamic-adar Index: 
Current Threshold Percentile =  70
Current Threshold Value =  0.5138983423697507
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24840 1705 854
False Positive Rate:  0.4991202346041056
False Positive Rate:  0.9656199677938808
Accuracy:  0.03438003220611916
----------------------------------------------------------------------------
0.013888888888888888
Jaccard_coefficient: 
Current Threshold Percentile =  70
Current Threshold Value =  0.2
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24840 1678 637
False Positive Rate:  0.6203814064362336
False Positive Rate:  0.9743558776167471
Accuracy:  0.025644122383252817
----------------------------------------------------------------------------

*********

24825 4439 1676
False Positive Rate:  0.6224374859202523
False Positive Rate:  0.9324874118831823
Accuracy:  0.06751258811681772
----------------------------------------------------------------------------
0.2749075777424936
Adamic-adar Index: 
Current Threshold Percentile =  30
Current Threshold Value =  0.32351545314867247
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24825 4439 1676
False Positive Rate:  0.6224374859202523
False Positive Rate:  0.9324874118831823
Accuracy:  0.06751258811681772
----------------------------------------------------------------------------
0.0136986301369863
Jaccard_coefficient: 
Current Threshold Percentile =  30
Current Threshold Value =  0.08333333333333333
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24825 4521 1496
False Positive Rate:  0.6690997566909975
False Positive Rate:  0.9397381671701913
Accuracy:  0.06026183282980866
-----------------------------------------------------------------------

24825 1293 668
False Positive Rate:  0.4833720030935808
False Positive Rate:  0.973091641490433
Accuracy:  0.02690835850956697
----------------------------------------------------------------------------
0.0136986301369863
Jaccard_coefficient: 
Current Threshold Percentile =  80
Current Threshold Value =  0.25
number_edge_in_test,number_edge_in_prediction,number_edge_in_both are: 
24825 1153 438
False Positive Rate:  0.6201214223764093
False Positive Rate:  0.982356495468278
Accuracy:  0.017643504531722056
----------------------------------------------------------------------------

*************************************************************************************
_____________________________________________________________________________________
_____________________________________________________________________________________
